# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f248c359940>
├── 'a' --> tensor([[-0.8953, -0.7000,  0.8721],
│                   [-1.1243, -0.3469, -0.6253]])
└── 'x' --> <FastTreeValue 0x7f248c359e20>
    └── 'c' --> tensor([[ 0.1127, -1.0834, -0.8514,  1.7591],
                        [-0.0364,  0.6869,  0.7473,  0.9332],
                        [-0.4081,  0.5905, -1.7458, -2.5872]])

In [4]:
t.a

tensor([[-0.8953, -0.7000,  0.8721],
        [-1.1243, -0.3469, -0.6253]])

In [5]:
%timeit t.a

89.1 ns ± 1.3 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f248c359940>
├── 'a' --> tensor([[ 0.8662,  0.0375, -1.4062],
│                   [-1.2433, -0.2751,  0.0101]])
└── 'x' --> <FastTreeValue 0x7f248c359e20>
    └── 'c' --> tensor([[ 0.1127, -1.0834, -0.8514,  1.7591],
                        [-0.0364,  0.6869,  0.7473,  0.9332],
                        [-0.4081,  0.5905, -1.7458, -2.5872]])

In [7]:
%timeit t.a = new_value

94.6 ns ± 1.26 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8953, -0.7000,  0.8721],
               [-1.1243, -0.3469, -0.6253]]),
    x: Batch(
           c: tensor([[ 0.1127, -1.0834, -0.8514,  1.7591],
                      [-0.0364,  0.6869,  0.7473,  0.9332],
                      [-0.4081,  0.5905, -1.7458, -2.5872]]),
       ),
)

In [10]:
b.a

tensor([[-0.8953, -0.7000,  0.8721],
        [-1.1243, -0.3469, -0.6253]])

In [11]:
%timeit b.a

70.9 ns ± 1.45 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2054,  2.1105,  1.5955],
               [ 1.9820, -1.6768, -0.6348]]),
    x: Batch(
           c: tensor([[ 0.1127, -1.0834, -0.8514,  1.7591],
                      [-0.0364,  0.6869,  0.7473,  0.9332],
                      [-0.4081,  0.5905, -1.7458, -2.5872]]),
       ),
)

In [13]:
%timeit b.a = new_value

662 ns ± 7.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.08 µs ± 13.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.5 µs ± 91.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

288 µs ± 6.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

273 µs ± 5.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f23dc280400>
├── 'a' --> tensor([[[-0.8953, -0.7000,  0.8721],
│                    [-1.1243, -0.3469, -0.6253]],
│           
│                   [[-0.8953, -0.7000,  0.8721],
│                    [-1.1243, -0.3469, -0.6253]],
│           
│                   [[-0.8953, -0.7000,  0.8721],
│                    [-1.1243, -0.3469, -0.6253]],
│           
│                   [[-0.8953, -0.7000,  0.8721],
│                    [-1.1243, -0.3469, -0.6253]],
│           
│                   [[-0.8953, -0.7000,  0.8721],
│                    [-1.1243, -0.3469, -0.6253]],
│           
│                   [[-0.8953, -0.7000,  0.8721],
│                    [-1.1243, -0.3469, -0.6253]],
│           
│                   [[-0.8953, -0.7000,  0.8721],
│                    [-1.1243, -0.3469, -0.6253]],
│           
│                   [[-0.8953, -0.7000,  0.8721],
│                    [-1.1243, -0.3469, -0.6253]]])
└── 'x' --> <FastTreeValue 0x7f23d565cca0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

49.3 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f23dc280370>
├── 'a' --> tensor([[-0.8953, -0.7000,  0.8721],
│                   [-1.1243, -0.3469, -0.6253],
│                   [-0.8953, -0.7000,  0.8721],
│                   [-1.1243, -0.3469, -0.6253],
│                   [-0.8953, -0.7000,  0.8721],
│                   [-1.1243, -0.3469, -0.6253],
│                   [-0.8953, -0.7000,  0.8721],
│                   [-1.1243, -0.3469, -0.6253],
│                   [-0.8953, -0.7000,  0.8721],
│                   [-1.1243, -0.3469, -0.6253],
│                   [-0.8953, -0.7000,  0.8721],
│                   [-1.1243, -0.3469, -0.6253],
│                   [-0.8953, -0.7000,  0.8721],
│                   [-1.1243, -0.3469, -0.6253],
│                   [-0.8953, -0.7000,  0.8721],
│                   [-1.1243, -0.3469, -0.6253]])
└── 'x' --> <FastTreeValue 0x7f23dc280430>
    └── 'c' --> tensor([[ 0.1127, -1.0834, -0.8514,  1.7591],
                        [-0.0364,  0.6869,  0.7473,  0.9332],
                 

In [23]:
%timeit t_cat(trees)

44.9 µs ± 677 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

88.5 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.1127, -1.0834, -0.8514,  1.7591],
                       [-0.0364,  0.6869,  0.7473,  0.9332],
                       [-0.4081,  0.5905, -1.7458, -2.5872]],
              
                      [[ 0.1127, -1.0834, -0.8514,  1.7591],
                       [-0.0364,  0.6869,  0.7473,  0.9332],
                       [-0.4081,  0.5905, -1.7458, -2.5872]],
              
                      [[ 0.1127, -1.0834, -0.8514,  1.7591],
                       [-0.0364,  0.6869,  0.7473,  0.9332],
                       [-0.4081,  0.5905, -1.7458, -2.5872]],
              
                      [[ 0.1127, -1.0834, -0.8514,  1.7591],
                       [-0.0364,  0.6869,  0.7473,  0.9332],
                       [-0.4081,  0.5905, -1.7458, -2.5872]],
              
                      [[ 0.1127, -1.0834, -0.8514,  1.7591],
                       [-0.0364,  0.6869,  0.7473,  0.9332],
                       [-0.4081,  0.5905, -1.7458, -2.5872]],

In [26]:
%timeit Batch.stack(batches)

108 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.1127, -1.0834, -0.8514,  1.7591],
                      [-0.0364,  0.6869,  0.7473,  0.9332],
                      [-0.4081,  0.5905, -1.7458, -2.5872],
                      [ 0.1127, -1.0834, -0.8514,  1.7591],
                      [-0.0364,  0.6869,  0.7473,  0.9332],
                      [-0.4081,  0.5905, -1.7458, -2.5872],
                      [ 0.1127, -1.0834, -0.8514,  1.7591],
                      [-0.0364,  0.6869,  0.7473,  0.9332],
                      [-0.4081,  0.5905, -1.7458, -2.5872],
                      [ 0.1127, -1.0834, -0.8514,  1.7591],
                      [-0.0364,  0.6869,  0.7473,  0.9332],
                      [-0.4081,  0.5905, -1.7458, -2.5872],
                      [ 0.1127, -1.0834, -0.8514,  1.7591],
                      [-0.0364,  0.6869,  0.7473,  0.9332],
                      [-0.4081,  0.5905, -1.7458, -2.5872],
                      [ 0.1127, -1.0834, -0.8514,  1.7591],
                   

In [28]:
%timeit Batch.cat(batches)

198 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

568 µs ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
